# Question 2: Histogram and Histogram Equalization

This notebook demonstrates image histogram computation, visualization, interpretation, and histogram equalization for contrast enhancement.

## Environment Setup

In [ ]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# Set image path (update this with your image path)
image_path = 'q1_and_2_picture.jpg'
if not os.path.isfile(image_path):
    raise ValueError(f"{image_path} does not exist in your computer.\n"
                     f"Replace the image_path variable with a path to an image that exists in your file system")

## Task 1: Compute the Histogram of a Grayscale Image

A histogram shows the distribution of pixel intensity values in an image. For a grayscale image, we count how many pixels have each intensity value from 0 to 255.

### Task 1a: Read Image and Convert to Grayscale

In [ ]:
# Read the image and convert to grayscale
original_image = cv2.imread(image_path)
if original_image is None:
    raise ValueError(f"Could not read image from {image_path}")

# Convert BGR to RGB for display
original_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

# Convert to grayscale
grayscale_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

print(f"Image shape: {original_image.shape}")
print(f"Grayscale shape: {grayscale_image.shape}")
print(f"Grayscale dtype: {grayscale_image.dtype}")
print(f"Pixel value range: [{grayscale_image.min()}, {grayscale_image.max()}]")

In [ ]:
# Display the grayscale image
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(original_rgb)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(grayscale_image, cmap='gray')
plt.title('Grayscale Image')
plt.axis('off')
plt.tight_layout()
plt.show()

### Task 1b: Compute Histogram Manually Using NumPy

We manually count how many pixels have each intensity value from 0 to 255.

In [ ]:
def compute_histogram_manual(image):
    """
    Compute histogram manually by counting pixels for each intensity value.
    
    Args:
        image: Grayscale image (2D numpy array with values 0-255)
    
    Returns:
        histogram: Array of size 256 containing pixel counts for each intensity
    """
    # Initialize histogram array with zeros
    histogram = np.zeros(256, dtype=np.int64)
    
    # Flatten the image to 1D for easier iteration
    flat_image = image.flatten()
    
    # Count pixels for each intensity value (0 to 255)
    for pixel_value in flat_image:
        histogram[pixel_value] += 1
    
    return histogram

In [ ]:
# Compute histogram manually
manual_histogram = compute_histogram_manual(grayscale_image)

# Display some sample counts
print("Manual Histogram Computation Results:")
print(f"  Pixels with value 0 (black): {manual_histogram[0]}")
print(f"  Pixels with value 1: {manual_histogram[1]}")
print(f"  Pixels with value 127 (mid-gray): {manual_histogram[127]}")
print(f"  Pixels with value 128: {manual_histogram[128]}")
print(f"  Pixels with value 254: {manual_histogram[254]}")
print(f"  Pixels with value 255 (white): {manual_histogram[255]}")
print(f"\nTotal pixel count: {manual_histogram.sum()}")
print(f"Expected total (height x width): {grayscale_image.shape[0] * grayscale_image.shape[1]}")

### Task 1c: Comparison with OpenCV's Built-in Function

In [ ]:
# Compute histogram using OpenCV
opencv_histogram = cv2.calcHist([grayscale_image], [0], None, [256], [0, 256])

# Flatten OpenCV histogram for comparison (it returns a 2D array)
opencv_histogram_flat = opencv_histogram.flatten()

# Verify that both methods produce the same result
are_equal = np.allclose(manual_histogram, opencv_histogram_flat)
print(f"Manual histogram matches OpenCV histogram: {are_equal}")

# Show difference if any
if not are_equal:
    diff = np.abs(manual_histogram - opencv_histogram_flat)
    print(f"Maximum difference: {diff.max()}")
else:
    print("Both methods produce identical results!")

## Task 2: Plot the Histogram

Visualize the histogram with properly labeled axes.

In [ ]:
def plot_histogram(histogram, title="Histogram of the Original Image", color='steelblue'):
    """
    Plot a histogram with proper labels.
    
    Args:
        histogram: Array of size 256 with pixel counts
        title: Title for the plot
        color: Color for the histogram bars
    """
    plt.figure(figsize=(12, 5))
    
    # x-axis: pixel intensities (0-255)
    x = np.arange(256)
    
    # Plot as bar chart
    plt.bar(x, histogram, color=color, width=1.0, edgecolor='none')
    
    # Label axes clearly
    plt.xlabel('Pixel Intensity (0-255)', fontsize=12)
    plt.ylabel('Number of Pixels', fontsize=12)
    plt.title(title, fontsize=14)
    
    # Set axis limits
    plt.xlim([0, 255])
    plt.ylim([0, histogram.max() * 1.1])  # Add 10% margin at top
    
    # Add grid for readability
    plt.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot the histogram of the original grayscale image
plot_histogram(manual_histogram, title="Histogram of the Original Image")

## Task 3: Interpretation Questions

Understanding what different histogram shapes tell us about an image.

### 3a. What does it mean when the histogram is concentrated on the left side?

**Answer:**

When the histogram is concentrated on the left side (low intensity values, 0-127), it means:
- The image is **dark** (underexposed)
- Most pixels have low intensity values (closer to black)
- The image has poor visibility in bright areas
- This often occurs in:
  - Night photography
  - Underexposed photos
  - Low-light conditions
  - Images with dark subject matter

Such images often benefit from histogram equalization or brightness adjustment to improve visibility.

### 3b. What does it mean when the histogram is stretched across all values?

**Answer:**

When the histogram is stretched across all values (0-255), it means:
- The image has **good contrast**
- The image uses the full range of available intensities
- There are both dark and bright regions in the image
- Details are visible in shadows, midtones, and highlights
- This is generally the **ideal histogram** for most images

A well-distributed histogram indicates:
- Proper exposure
- Good dynamic range utilization
- Balanced lighting conditions

### 3c. What do high peaks represent?

**Answer:**

High peaks in a histogram represent:
- **Dominant intensity values** - many pixels share the same or similar intensity
- **Large uniform regions** - areas with consistent color/brightness
- Common causes include:
  - **Background regions**: Large uniform backgrounds (sky, walls, etc.)
  - **Foreground objects**: Dominant colored objects
  - **Overexposed areas**: Peak at 255 (clipped highlights)
  - **Underexposed areas**: Peak at 0 (clipped shadows)

For example:
- A peak at 255 might indicate overexposed/blown-out highlights
- A peak at 0 might indicate crushed/blocked shadows
- A peak at mid-values might indicate a large gray area in the image

## Task 4: Histogram Equalization

Histogram equalization enhances image contrast by redistributing pixel intensities to achieve a more uniform histogram distribution.

### Task 4a-b: Manual Implementation Using CDF Method

In [ ]:
def histogram_equalization_manual(image):
    """
    Perform histogram equalization manually using the CDF method.
    
    The algorithm:
    1. Compute the histogram of the image
    2. Compute the Cumulative Distribution Function (CDF)
    3. Normalize the CDF to map to [0, 255]
    4. Use the normalized CDF as a lookup table to transform pixel values
    
    Args:
        image: Grayscale image (2D numpy array)
    
    Returns:
        equalized: Histogram-equalized image
    """
    # Step 1: Compute histogram
    histogram = compute_histogram_manual(image)
    
    # Step 2: Compute Cumulative Distribution Function (CDF)
    cdf = histogram.cumsum()
    
    # Step 3: Normalize CDF to [0, 255]
    # Formula: cdf_normalized = (cdf - cdf_min) * 255 / (total_pixels - cdf_min)
    cdf_min = cdf[cdf > 0].min()  # Minimum non-zero CDF value
    total_pixels = image.size
    
    # Create lookup table
    cdf_normalized = ((cdf - cdf_min) * 255) / (total_pixels - cdf_min)
    cdf_normalized = np.round(cdf_normalized).astype(np.uint8)
    
    # Step 4: Apply transformation using lookup table
    equalized = cdf_normalized[image]
    
    return equalized

In [ ]:
# Apply manual histogram equalization
equalized_manual = histogram_equalization_manual(grayscale_image)

# Apply OpenCV's histogram equalization for comparison
equalized_opencv = cv2.equalizeHist(grayscale_image)

# Verify both methods produce similar results
difference = np.abs(equalized_manual.astype(np.int16) - equalized_opencv.astype(np.int16))
print(f"Maximum difference between manual and OpenCV: {difference.max()}")
print(f"Average difference: {difference.mean():.4f}")

### Task 4c: Display Original and Equalized Images with Histograms

In [ ]:
# Compute histograms for both images
histogram_original = compute_histogram_manual(grayscale_image)
histogram_equalized = compute_histogram_manual(equalized_manual)

# Create a comprehensive comparison figure
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Original image
axes[0, 0].imshow(grayscale_image, cmap='gray')
axes[0, 0].set_title('Original Image', fontsize=12)
axes[0, 0].axis('off')

# Equalized image
axes[0, 1].imshow(equalized_manual, cmap='gray')
axes[0, 1].set_title('Equalized Image', fontsize=12)
axes[0, 1].axis('off')

# Original histogram
axes[1, 0].bar(np.arange(256), histogram_original, color='steelblue', width=1.0)
axes[1, 0].set_xlabel('Pixel Intensity (0-255)', fontsize=10)
axes[1, 0].set_ylabel('Number of Pixels', fontsize=10)
axes[1, 0].set_title('Histogram of Original Image', fontsize=12)
axes[1, 0].set_xlim([0, 255])
axes[1, 0].grid(axis='y', alpha=0.3)

# Equalized histogram
axes[1, 1].bar(np.arange(256), histogram_equalized, color='coral', width=1.0)
axes[1, 1].set_xlabel('Pixel Intensity (0-255)', fontsize=10)
axes[1, 1].set_ylabel('Number of Pixels', fontsize=10)
axes[1, 1].set_title('Histogram of Equalized Image', fontsize=12)
axes[1, 1].set_xlim([0, 255])
axes[1, 1].grid(axis='y', alpha=0.3)

plt.suptitle('Histogram Equalization: Before and After', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Task 4d: Explanation of How Equalization Changes Pixel Distribution

**How Histogram Equalization Works:**

Histogram equalization transforms the pixel intensities so that the output histogram is approximately uniform (flat). Here's what happens:

1. **Compute CDF**: The Cumulative Distribution Function (CDF) is computed from the histogram. The CDF represents the cumulative probability of pixel intensities.

2. **Normalize CDF**: The CDF is normalized to map values to the range [0, 255].

3. **Transform Pixels**: Each pixel value is replaced by its corresponding CDF value, which acts as a lookup table.

**Effects on Pixel Distribution:**

- **Before equalization**: Pixels may be clustered in certain intensity ranges (narrow histogram)
- **After equalization**: Pixels are spread more evenly across all intensity values (wider histogram)

**Mathematical Interpretation:**

The transformation $T(r) = (L-1) \times CDF(r)$ where:
- $r$ is the original pixel intensity
- $L$ is the number of intensity levels (256)
- $CDF(r)$ is the cumulative distribution function

This mapping ensures that pixels are redistributed to utilize the full dynamic range, enhancing contrast.

In [ ]:
# Visualize the CDF transformation
histogram_orig = compute_histogram_manual(grayscale_image)
cdf_original = histogram_orig.cumsum()
cdf_normalized = cdf_original / cdf_original.max()  # Normalize to [0, 1]

histogram_eq = compute_histogram_manual(equalized_manual)
cdf_equalized = histogram_eq.cumsum()
cdf_eq_normalized = cdf_equalized / cdf_equalized.max()

# Plot CDFs
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(cdf_normalized, color='steelblue', linewidth=2)
plt.xlabel('Pixel Intensity (0-255)', fontsize=10)
plt.ylabel('Cumulative Probability', fontsize=10)
plt.title('CDF of Original Image', fontsize=12)
plt.grid(alpha=0.3)
plt.xlim([0, 255])

plt.subplot(1, 2, 2)
plt.plot(cdf_eq_normalized, color='coral', linewidth=2)
plt.xlabel('Pixel Intensity (0-255)', fontsize=10)
plt.ylabel('Cumulative Probability', fontsize=10)
plt.title('CDF of Equalized Image', fontsize=12)
plt.grid(alpha=0.3)
plt.xlim([0, 255])

plt.suptitle('CDF Comparison: The equalized CDF is more linear (uniform distribution)', fontsize=11)
plt.tight_layout()
plt.show()

print("Note: The CDF of the equalized image is approximately linear (diagonal),")
print("indicating a more uniform distribution of pixel intensities.")

## Task 5: Comparison and Analysis

A detailed comparison of when histogram equalization is beneficial and when it may cause issues.

### 5a.i. When does histogram equalization improve images?

**Histogram equalization improves images when:**

1. **Low contrast images**: Images where most pixels are clustered in a narrow intensity range
   - Dark/underexposed images (histogram concentrated on left)
   - Bright/overexposed images (histogram concentrated on right)
   - Washed-out images (histogram in middle with no extremes)

2. **Images with poor lighting conditions**:
   - Photos taken in fog, haze, or poor visibility
   - Images with uniform lighting that lacks definition

3. **Scientific and technical imaging**:
   - Satellite imagery
   - Microscopy images
   - Images where extracting details is more important than natural appearance

4. **Pre-processing for computer vision tasks**:
   - Feature detection
   - Edge detection
   - Object recognition

### 5a.ii. When does histogram equalization create noise or distort colors?

**Histogram equalization can cause problems when:**

1. **Images with already good contrast**: Applying equalization to well-exposed images can:
   - Over-enhance contrast making the image look unnatural
   - Introduce posterization effects

2. **Images with large uniform areas**:
   - Areas that should be uniform become noisy
   - Subtle variations get amplified into visible artifacts

3. **Noise amplification**:
   - In dark regions, noise gets amplified along with the signal
   - Low-quality images become grainier

4. **Color images (when applied incorrectly)**:
   - Applying equalization to RGB channels independently distorts colors
   - Can cause color shifts and unnatural tones
   - For color images, equalization should be applied to the luminance channel only (in HSV, LAB, or YCbCr color spaces)

5. **Images with intentional lighting effects**:
   - Artistic images where shadows/highlights are intentional
   - Can destroy the intended mood or aesthetic

### 5a.iii. Why is histogram equalization often used on medical or low-light images?

**Medical Imaging Applications:**

1. **X-ray images**: Often have limited contrast due to the nature of X-ray absorption
   - Equalization helps reveal subtle differences in tissue density
   - Makes fractures, tumors, and abnormalities more visible

2. **CT scans and MRI**: Improve visibility of anatomical structures
   - Better differentiation between tissue types
   - Enhanced boundary detection for diagnosis

3. **Ultrasound images**: Typically have poor contrast
   - Equalization helps distinguish between different tissue types

**Low-Light Imaging Applications:**

1. **Night photography**: Images captured with limited light have compressed dynamic range
   - Equalization expands the range to show hidden details

2. **Security/surveillance footage**: Often captured in poor lighting
   - Enhanced contrast helps identify subjects and activities

3. **Astronomical imaging**: Faint celestial objects require contrast enhancement
   - Reveals stars, galaxies, and nebulae that would otherwise be invisible

**Key Reason:**

In both medical and low-light scenarios, **extracting information is more important than aesthetic quality**. The goal is to make previously invisible details visible, even if it introduces some artifacts. The enhanced visibility can be critical for:
- Medical diagnosis (potentially life-saving)
- Security analysis
- Scientific discovery

In [ ]:
# Summary statistics comparison
print("="*60)
print("Summary: Image Statistics Before and After Equalization")
print("="*60)
print(f"\n{'Metric':<25} {'Original':>15} {'Equalized':>15}")
print("-"*60)
print(f"{'Mean intensity':<25} {grayscale_image.mean():>15.2f} {equalized_manual.mean():>15.2f}")
print(f"{'Standard deviation':<25} {grayscale_image.std():>15.2f} {equalized_manual.std():>15.2f}")
print(f"{'Minimum value':<25} {grayscale_image.min():>15} {equalized_manual.min():>15}")
print(f"{'Maximum value':<25} {grayscale_image.max():>15} {equalized_manual.max():>15}")
print(f"{'Value range':<25} {grayscale_image.max()-grayscale_image.min():>15} {equalized_manual.max()-equalized_manual.min():>15}")
print("="*60)
print("\nNote: Higher standard deviation indicates better contrast (more spread).")
print("Equalization typically increases standard deviation and expands value range.")